# <span style="color:purple">Script edit dot</span>

In [157]:
import pydot
from pydot import Edge
from pydot import Node
import re
global subgraphs
subgraphs={}

#### Import a graph from dot and return the pydot graph

In [89]:
def graph_from_dot(file):
    """
       Import a graph from dot and return the pydot graph
       
       :param file: path to the file to import
       :return: pydot graph
       :rtype: Dot
    """
    (graph, ) = pydot.graph_from_dot_file(str(file))
    return graph

In [90]:
G = graph_from_dot("../Tests/while_with_foo/test.c.011t.cfg.dot");

#### Extract subgraphs, edges, nodes

In [91]:
def extract_subgraphs(subgraph_list):
    global subgraphs
    """
       Extract subgraphs from a list of subgraphs
       
       :param graph: graph or subgraph from which subgraphs are to be extracted
       :return: List of subgraphs
       :rtype: List
    """
    new_subgraphs = []
    for subgraph in subgraph_list:
        subgraph.set_name(subgraph.get_name().strip("\"").replace("cluster_", ""))
        subgraphs[subgraph.get_name()] = subgraph
        new_subgraphs += subgraph.get_subgraph_list()
    if(new_subgraphs == []):
        return
    else:
        return extract_subgraphs(new_subgraphs)

In [92]:
extract_subgraphs([G])
print(subgraphs)

{'test.c.011t.cfg': <pydot.Dot object at 0x7faceb3f3f98>, 'foo': <pydot.Subgraph object at 0x7faceb275eb8>, 'main': <pydot.Subgraph object at 0x7faceb564f98>, '1_1': <pydot.Subgraph object at 0x7faceb278160>}


In [93]:
def extract_nodes(graph):
    """
       Extract nodes from a graph or subgraph
       
       :param graph: graph or subgraph from which nodes are to be extracted
       :return: List of nodes
       :rtype: List
    """
    return graph.get_nodes()

In [95]:
nodes_main = extract_nodes(subgraphs['foo'])
print(nodes_main)

[<pydot.Node object at 0x7faceb272cf8>, <pydot.Node object at 0x7faceb272d30>, <pydot.Node object at 0x7faceb272f98>, <pydot.Node object at 0x7faceb25bfd0>, <pydot.Node object at 0x7faceb275128>, <pydot.Node object at 0x7faceb28ddd8>]


In [96]:
def extract_edges(graph):
    """
       Extract edges from a graph or subgraph
       
       :param graph: graph or subgraph from which edges are to be extracted
       :return: List of edges
       :rtype: List
    """
    return graph.get_edges()

In [98]:
edges_main = extract_edges(subgraphs['main'])
print(edges_main)

[<pydot.Edge object at 0x7faceb2ad1d0>, <pydot.Edge object at 0x7faceb2763c8>, <pydot.Edge object at 0x7faceb24ccf8>, <pydot.Edge object at 0x7faceb25b710>, <pydot.Edge object at 0x7faceb5a4860>, <pydot.Edge object at 0x7faceb5a30b8>, <pydot.Edge object at 0x7faceb613cc0>, <pydot.Edge object at 0x7faceb2a6550>, <pydot.Edge object at 0x7faceb3da828>, <pydot.Edge object at 0x7faceb5d4e48>, <pydot.Edge object at 0x7faceb563668>]


#### Add an edge, a node

In [99]:
def add_edge(graph, tail_node, head_node, label="[0%]", style="solid, bold", color="black"):
    """
       Add an edge to a graph or subgraph
       
       :param graph: graph or subgraph to which the edge is added
       :param tail_node: origin of the edge
       :param head_node: destination of the edge
    """
    edge = Edge(tail_node.get_name()+":s", head_node.get_name()+":n", label=label, style=style, color=color)
    graph.add_edge(edge)
    return

In [103]:
add_edge(subgraphs['main'], nodes_main[0], nodes_main[1], color="red")

In [117]:
def add_node(graph, node_name, label, shape='record', style='filled', fillcolor='lightgrey'):
    """
       Add a node to a graph or subgraph
       
       :param graph: graph or subgraph to which the node is added
       :param node_name: name of the node
       :param label: label of the node
       :param shape: shape of the node (default "record")
       :param style: style of the node (default "field")
       :param fillcolor: color of the node (default "lightgrey")
    """
    node = Node(name=node_name, shape=shape, style=style, fillcolor=fillcolor, label=label)
    graph.add_node(node)
    return

In [118]:
add_node(G, 'node_yolo', 'yolo')

#### Create nodes based on the function calls

In [172]:
def analyse_label(node):
    """
        Find if it exists, the first call to a function in a node
        
        :param node: the node examined
        :return: tuple containing the name of the function and its position in the label of the node
        :rtype: tuple
    """
    global subgraphs
    for match in re.finditer("\w+\\\s\(\w*\)", node.get_attributes['label']):
        function_name = re.search("\w+", match.group())
        if function_name is not None:
            if function_name in subgraphs:
                return (function_name, match.group())
    return (None, None)

In [ ]:
def create_labels(node, function_call):
    """
        Create the labels for the new node, and for the node cut in half.
        
        :param node: the node cut
        :param function_call: the string containing the function call used as a regex expression
        :return: tuple containing the new label for the node cut, the label for the new node, and the value of
                 the cut node's bb
        :rtype: tuple
    """
    label_node = node.get_attributes["label"]
    
    match_function = re.search('\\l\\(.(?!\\l))*' + function_call + ';\\l\\', label_node)
    
    bb_node1 = re.search('\d+', re.search('<bb\ \\d+\>', label_node).group()).group()
    label_node1 = label_node[0:match_function.start()] + '\l\|call\ ' + function_call + ';\}'
    
    match_variable = re.search('\w+\\ =\ ' + function_call, label_node)
    label_node2 = '{ FREQ:0 |\<bb\ ' + bb_node1 + '\>:\l\|'
    if match_variable is not None:
        variable = re.search('\w+', match_variable.group()).group()
        label_node2 += variable + '\ =\ '
    label_node2 += 'return\ ' + function_call + ';\\l\\' + label_node[match_function.end() + 1 :]
    
    return (label_node1, label_node2, bb_node1)
    
    

In [ ]:
def update_node(node, attribute, value):
    node.set(attribute, value)
    return

In [ ]:
def update_nodes_after_bb(nodes, bb):
    

On veut récupérer les noeuds qui pourront être update ce qui veut dire qu'il faut itérer dans les sous-graphes enfants et parents

In [113]:
def main(file):
    global subgraphs
    graph = graph_from_dot(file)
    subgraphs = {}
    extract_subgraphs([graph])
    for (name, subgraph) in subgraphs.items():
        nodes = extract_nodes(subgraph)
        for node in nodes:
            (name_function, result) = analyse_label(node)
            if name_function is not None:
                (label_node1, label_node2, bb_node1) = create_labels(node, result)
                update_nodes_after_bb(nodes)
                update_node()
                
                
                
        
    
    
    

In [114]:
main("../Tests/while_with_foo/test.c.011t.cfg.dot")

{'shape': 'Mdiamond', 'style': 'filled', 'fillcolor': 'white', 'label': '"ENTRY"'}
{'shape': 'Mdiamond', 'style': 'filled', 'fillcolor': 'white', 'label': '"ENTRY"'}
{'shape': 'record', 'style': 'filled', 'fillcolor': 'lightgrey', 'label': '"{ FREQ:0 |\\<bb\\ 3\\>:\\l\\\n|_1\\ =\\ foo\\ (b);\\l\\\n|if\\ (_1\\ \\>\\ 3)\\l\\\n\\ \\ goto\\ \\<bb\\ 4\\>;\\ [0.00%]\\l\\\nelse\\l\\\n\\ \\ goto\\ \\<bb\\ 5\\>;\\ [0.00%]\\l\\\n}"'}


In [106]:
help(nodes_main[0])

Help on Node in module pydot object:

class Node(Common)
 |  Node(name='', obj_dict=None, **attrs)
 |  
 |  A graph node.
 |  
 |  This class represents a graph's node with all its attributes.
 |  
 |  node(name, attribute=value, ...)
 |  
 |  name: node's name
 |  
 |  All the attributes defined in the Graphviz dot language should
 |  be supported.
 |  
 |  Method resolution order:
 |      Node
 |      Common
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name='', obj_dict=None, **attrs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  add_style(self, style)
 |  
 |  get_name(self)
 |      Get the node's name.
 |  
 |  get_port(self)
 |      Get the node's port.
 |  
 |  set_name(self, node_name)
 |      Set the node's name.
 |  
 |  to_string(self)
 |      Return string representation of node in DOT language.
 |  
 |  --------------------------------------------------------

In [168]:


string = "The rain in Spain"
x = re.search("oiseau", string)

print("The first white-space character is located in position:") 

The first white-space character is located in position:


In [170]:
print(x)

None


In [145]:
help(re.compile)

Help on function compile in module re:

compile(pattern, flags=0)
    Compile a regular expression pattern, returning a Pattern object.



In [120]:
l = [1 , 2 , 3]
for i in l:
    if(i == 3):
        l.append(4)
    print(i)

1
2
3
4


SyntaxError: unexpected character after line continuation character (<ipython-input-136-733f30c2e64d>, line 1)